In [2]:
import tensorflow as tf
tf.get_logger().setLevel(40) # suppress deprecation messages
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.layers import Dense, Input, Embedding, Flatten, Concatenate, Reshape,Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from time import time
from alibi.datasets import fetch_adult
from alibi.explainers import CounterfactualProto

In [3]:
adult = fetch_adult()
data = adult.data
target = adult.target
feature_names = adult.feature_names
category_map_tmp = adult.category_map
target_names = adult.target_names

In [4]:
def set_seed(s=0):
    np.random.seed(s)
    tf.random.set_seed(s)

In [5]:
set_seed()
data_perm = np.random.permutation(np.c_[data, target])
X = data_perm[:,:-1]
y = data_perm[:,-1]

In [6]:
idx = 30000
y_train, y_test = y[:idx], y[idx:]

In [7]:
X = np.c_[X[:, 1:8], X[:, 11], X[:, 0], X[:, 8:11]]

In [8]:
feature_names = feature_names[1:8] + feature_names[11:12] + feature_names[0:1] + feature_names[8:11]
print(feature_names)

['Workclass', 'Education', 'Marital Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country', 'Age', 'Capital Gain', 'Capital Loss', 'Hours per week']


In [9]:
category_map = {}
for i, (_, v) in enumerate(category_map_tmp.items()):
    category_map[i] = v

In [10]:
cat_vars_ord = {}
n_categories = len(list(category_map.keys()))
for i in range(n_categories):
    cat_vars_ord[i] = len(np.unique(X[:, i]))
print(cat_vars_ord)


{0: 9, 1: 7, 2: 4, 3: 9, 4: 6, 5: 5, 6: 2, 7: 11}


In [11]:
X_num = X[:, -4:].astype(np.float32, copy=False)
xmin, xmax = X_num.min(axis=0), X_num.max(axis=0)
rng = (-1., 1.)
X_num_scaled = (X_num - xmin) / (xmax - xmin) * (rng[1] - rng[0]) + rng[0]
X_num_scaled_train = X_num_scaled[:idx, :]
X_num_scaled_test = X_num_scaled[idx+1:, :]

In [12]:
X = np.c_[X[:, :-4], X_num_scaled].astype(np.float32, copy=False)
X_train, X_test = X[:idx, :], X[idx:, :]
print(X_train.shape, X_test.shape)

(30000, 12) (2561, 12)


In [13]:
def nn_ord():
    x_in = Input(shape=(12,))
    layers_in = []
    # embedding layers
    for i, (_, v) in enumerate(cat_vars_ord.items()):
        emb_in = Lambda(lambda x: x[:, i:i+1]+1)(x_in)
        emb_dim = int(max(min(np.ceil(.5 * v), 50), 2))
        emb_layer = Embedding(input_dim=v+1, output_dim=emb_dim, input_length=1)(emb_in)
        emb_layer=Flatten()(emb_layer)
        layers_in.append(emb_layer)
    # numerical layers
    num_in = Lambda(lambda x: x[:, -4:])(x_in)
    num_layer = Dense(16)(num_in)
    layers_in.append(num_layer)
    # combine
    x = Concatenate()(layers_in)
    x = Dense(60, activation='relu')(x)
    x = Dropout(.2)(x)
    x = Dense(60, activation='relu')(x)
    x = Dropout(.2)(x)
    x = Dense(60, activation='relu')(x)
    x = Dropout(.2)(x)
    x_out = Dense(2, activation='softmax')(x)
    nn = Model(inputs=x_in, outputs=x_out)
    nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return nn

In [19]:
import numpy as np
aa=int(max(min(np.ceil(.5 * 9), 50), 2))

v=9
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(v+1, aa, input_length=1))
model.compile('rmsprop', 'mse')
output_array = model.predict(np.array([3]))
print(output_array)

[[[ 0.00022001 -0.01777273 -0.00655563  0.0315379   0.01513681]]]


In [20]:
set_seed()
nn = nn_ord()
nn.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 12)]         0           []                               
                                                                                                  
 lambda (Lambda)                (None, 1)            0           ['input_1[0][0]']                
                                                                                                  
 lambda_1 (Lambda)              (None, 1)            0           ['input_1[0][0]']                
                                                                                                  
 lambda_2 (Lambda)              (None, 1)            0           ['input_1[0][0]']                
                                                                                              

__________________________________________________________________________________________________


In [24]:
nn.fit(X_train, to_categorical(y_train), batch_size=128, epochs=30, verbose=1)

Train on 30000 samples
Epoch 1/30
30000/30000 [==============================] - 1s 32us/sample - loss: 0.4074 - acc: 0.8058
Epoch 2/30
30000/30000 [==============================] - 1s 25us/sample - loss: 0.3321 - acc: 0.8478
Epoch 3/30
30000/30000 [==============================] - 1s 28us/sample - loss: 0.3226 - acc: 0.8505
Epoch 4/30
30000/30000 [==============================] - 1s 27us/sample - loss: 0.3188 - acc: 0.8528
Epoch 5/30
30000/30000 [==============================] - 1s 25us/sample - loss: 0.3169 - acc: 0.8542
Epoch 6/30
30000/30000 [==============================] - 1s 26us/sample - loss: 0.3153 - acc: 0.8559
Epoch 7/30
30000/30000 [==============================] - 1s 27us/sample - loss: 0.3133 - acc: 0.8549
Epoch 8/30
30000/30000 [==============================] - 1s 29us/sample - loss: 0.3122 - acc: 0.8563
Epoch 9/30
30000/30000 [==============================] - 1s 26us/sample - loss: 0.3111 - acc: 0.8569
Epoch 10/30
30000/30000 [==============================] - 

In [15]:
score=nn.evaluate(X_test, to_categorical(y_test))
print(score[1])

0.86021084


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.


In [16]:
X = X_test[0].reshape((1,) + X_test[0].shape)

In [17]:
shape = X.shape
beta = .01
c_init = 1.
c_steps = 5
max_iterations = 500
rng = (-1., 1.) # scale features between -1 and 1
rng_shape = (1,) + data.shape[1:]
feature_range = ((np.ones(rng_shape) * rng[0]).astype(np.float32),
(np.ones(rng_shape) * rng[1]).astype(np.float32))

In [18]:
set_seed()
# define predict function
predict_fn = lambda x: nn.predict(x)
cf = CounterfactualProto(predict_fn,
     shape,
     beta=beta,
     cat_vars=cat_vars_ord,
     max_iterations=max_iterations,
     feature_range=feature_range,
     c_init=c_init,
     c_steps=c_steps,
     eps=(.01, .01) # perturbation size for numerical gradients
)

In [19]:
cf.fit(X_train, d_type='abdm', disc_perc=[25, 50, 75]);

In [20]:
set_seed()
explanation = cf.explain(X)

In [21]:
def describe_instance(X, explanation, eps=1e-2):
    print('Original instance: {} -- proba: {}'.format(target_names[explanation.orig_class],
          explanation.orig_proba[0]))
    print('Counterfactual instance: {} -- proba: {}'.format(target_names[explanation.cf['class']],
           explanation.cf['proba'][0]))
    print('\nCounterfactual perturbations...')
    print('\nCategorical:')
    X_orig_ord = X
    X_cf_ord = explanation.cf['X']
    delta_cat = {}
    for i, (_, v) in enumerate(category_map.items()):
        cat_orig = v[int(X_orig_ord[0, i])]
        cat_cf = v[int(X_cf_ord[0, i])]
        if cat_orig != cat_cf:
            delta_cat[feature_names[i]] = [cat_orig, cat_cf]
    if delta_cat:
        for k, v in delta_cat.items():
            print('{}: {} --> {}'.format(k, v[0], v[1]))
    print('\nNumerical:')
    delta_num = X_cf_ord[0, -4:] - X_orig_ord[0, -4:]
    n_keys = len(list(cat_vars_ord.keys()))
    for i in range(delta_num.shape[0]):
        if np.abs(delta_num[i]) > eps:
            print('{}: {:.2f} --> {:.2f}'.format(feature_names[i+n_keys],
                  X_orig_ord[0,i+n_keys],
                X_cf_ord[0,i+n_keys]))

In [22]:
describe_instance(X, explanation)

Original instance: <=50K -- proba: [0.8820062  0.11799379]
Counterfactual instance: >50K -- proba: [0.49965528 0.5003447 ]

Counterfactual perturbations...

Categorical:

Numerical:
Capital Gain: -0.96 --> -0.83
